In [5]:
import pandas as pd

# Step 1: Load the dataset with proper encoding
df = pd.read_csv('zomato.csv', encoding='latin1')

# Step 2: Normalize column names (lowercase, replace spaces with underscores)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Step 3: Drop duplicates
df.drop_duplicates(inplace=True)

# Step 4: Drop irrelevant columns if present
columns_to_drop = [
    'restaurant_id', 'country_code', 'address', 'locality_verbose',
    'longitude', 'latitude', 'has_table_booking', 'has_online_delivery',
    'is_delivering_now', 'switch_to_order_menu', 'rating_color', 'rating_text'
]
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

# Step 5: Rename columns for easier handling 
df.rename(columns={
    'average_cost_for_two': 'cost',
    'aggregate_rating': 'rating'
}, inplace=True)

# Step 6: Clean cost and rating columns
df['cost'] = pd.to_numeric(df['cost'], errors='coerce')
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Step 7: Fill missing numeric values with column-wise mean
df.fillna(df.mean(numeric_only=True), inplace=True)

# Step 8: Drop rows with missing critical text fields
df.dropna(subset=['cuisines', 'city'], inplace=True)

# Step 9: Normalize text columns
df['cuisines'] = df['cuisines'].str.lower().str.strip()
df['city'] = df['city'].str.lower().str.strip()
df['restaurant_name'] = df['restaurant_name'].str.strip()

# Step 10: Feature engineering

# Cost bucket
def cost_bucket(cost):
    if cost < 300:
        return 'Low'
    elif cost < 700:
        return 'Medium'
    else:
        return 'High'

df['cost_bucket'] = df['cost'].apply(cost_bucket)

# Extract primary cuisine
df['primary_cuisine'] = df['cuisines'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else x)

# Round rating
df['rating'] = df['rating'].round(1)

# Save cleaned data
# df.to_csv('zomato_cleaned.csv', index=False)

# Check result
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
Index: 9542 entries, 0 to 9550
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant_name  9542 non-null   object 
 1   city             9542 non-null   object 
 2   locality         9542 non-null   object 
 3   cuisines         9542 non-null   object 
 4   cost             9542 non-null   int64  
 5   currency         9542 non-null   object 
 6   price_range      9542 non-null   int64  
 7   rating           9542 non-null   float64
 8   votes            9542 non-null   int64  
 9   cost_bucket      9542 non-null   object 
 10  primary_cuisine  9542 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 894.6+ KB
None
          restaurant_name              city  \
0        Le Petit Souffle       makati city   
1        Izakaya Kikufuji       makati city   
2  Heat - Edsa Shangri-La  mandaluyong city   
3                    Ooma  mandaluyong city